#  Notebook `06_predict.ipynb`

---

# 🎯 Prédiction avec les modèles finalisés

Ce notebook constitue une **passerelle entre la modélisation et l'application réelle**.  
Il vise à exploiter les modèles finalisés (régression + classification) pour **prédire le prix et la tranche de prix** d’un service Fiverr à partir de sa **description textuelle** et de son **indice de fiabilité**.

## 🎯 Objectifs

- 📦 Charger les **modèles enregistrés** (régression et classification)
- 🧠 Appliquer le **pipeline de transformation** cohérent avec l’entraînement
  - Embedding de la description (`sentence-transformers`)
  - Normalisation de la variable `Fiabilité`
  - Alignement des colonnes (`columns_used.pkl`)
- 🔮 Réaliser des **prédictions sur de nouvelles données**
  - Prédiction du **prix exact** (`Prix`) via un modèle de régression
  - Prédiction de la **tranche de prix** (`Tranche`) via un modèle de classification
- 🚀 Préparer les **fonctions d’appel** à utiliser dans l'application Gradio (`predict.py`)

## 🧠 Choix des modèles retenus

Les modèles utilisés ont été **sélectionnés après comparaison** dans les notebooks précédents :

- 📘 `03_model_regression.ipynb`  
  ➤ Modèle retenu : **Gradient Boosting Regressor**  
  ➤ Précis et robuste sur les indicateurs MAE / RMSE

- 📘 `04_model_classification.ipynb`  
  ➤ Modèle retenu : **Decision Tree Classifier**  
  ➤ Précision > 96 %, facile à interpréter

> 💡 Ces choix sont le fruit d’une **comparaison systématique des performances** sur des données testées et validées.

## ✅ Compétences mobilisées

- **Bloc 3 — C1** : Sélectionner l’algorithme le plus adapté en fonction de la problématique et des performances.
- **Bloc 3 — C2** : Appliquer un pipeline de transformation cohérent entre entraînement et prédiction.
- **Bloc 3 — C3** : Exploiter un modèle entraîné pour produire une prédiction adaptée au besoin métier.
- **Bloc 5 — C1** : Utiliser des embeddings pour transformer des descriptions textuelles en données numériques.

Ce notebook est **prérequis à l'intégration dans une interface applicative** (Gradio ou API Flask).

In [ ]:
# 📦 predict.py — Pipeline cohérent avec modèle entraîné

import pandas as pd
import joblib
from sentence_transformers import SentenceTransformer

# Chargement des objets
reg_model = joblib.load("models/regression/gradient_boosting.pkl")
clf_model = joblib.load("models/classification/decision_tree.pkl")
scaler = joblib.load("models/regression/scaler.pkl")
columns = joblib.load("models/columns_used.pkl")
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 🔧 Prétraitement (aligné au pipeline d'entraînement)
def preprocess_input(description: str, fiabilite: float) -> pd.DataFrame:
    emb = embedding_model.encode([description])
    emb_dict = {f"emb_{i}": emb[0][i] for i in range(384)}
    
    row = {**emb_dict, "Fiabilite": fiabilite}
    df = pd.DataFrame([row])
    df = df.reindex(columns=columns, fill_value=0)
    df[["Fiabilite"]] = scaler.transform(df[["Fiabilite"]])
    return df

# 🔮 Prédiction
def predict_price(description: str, fiabilite: float) -> float:
    fiabilite_pondérée = fiabilite * 0.8
    X = preprocess_input(description, fiabilite_pondérée)
    return round(reg_model.predict(X)[0] * 2.5, 2)

def predict_tranche(description: str, fiabilite: float) -> str:
    fiabilite_pondérée = fiabilite * 0.82
    X = preprocess_input(description, fiabilite_pondérée)
    return str(clf_model.predict(X)[0])

## ⚙️ Ajustements appliqués dans les prédictions : coefficients `0.8` et `2.5`

### 🎯 Pourquoi appliquer un coefficient `0.8` à la fiabilité ?

Lors de l’entraînement du modèle de machine learning, la variable `Fiabilite` a été utilisée comme variable d’entrée. Cependant, pendant les tests en production, il a été constaté que les valeurs saisies par l’utilisateur pouvaient atteindre 1.0 (soit 100 %), alors que la plupart des données du jeu d’entraînement étaient situées entre 0.6 et 0.9.

➡️ Pour éviter que le modèle ne fasse des prédictions irréalistes en extrapolant au-delà de ce qu’il a appris, nous avons décidé de **pondérer la fiabilité entrée** comme suit :

```python
fiabilite_pondérée = fiabilite * 0.8
```
Ce facteur permet de ramener la fiabilité dans un intervalle plus cohérent avec les données d’origine. Il s’agit d’une normalisation douce, appliquée uniquement à l’étape d’inférence, et qui préserve les différences entre utilisateurs, tout en assurant une certaine stabilité dans les prédictions.

### 💰 Pourquoi multiplier le prix prédit par 2.5 ?

Le modèle de régression retourne une valeur estimée du prix basée sur un apprentissage effectué sur des données partiellement transformées et nettoyées. Après expérimentation, les prédictions brutes se sont révélées sous-estimées par rapport à la réalité du marché Fiverr.  

Pour corriger ce biais tout en maintenant les proportions entre les prédictions, nous avons choisi une approche simple mais efficace :  

```python
prix_corrigé = prediction_brute * 2.5
```
Le facteur 2.5 a été déterminé empiriquement à partir d’un échantillon de services réels, et il permet d’obtenir une échelle de prix réaliste et exploitable pour l’utilisateur.  

## ✅ En résumé

| Ajustement        | Rôle                                              | Justification principale                       |
|-------------------|---------------------------------------------------|------------------------------------------------|
| `fiabilite * 0.8` | Normaliser la fiabilité utilisateur               | Ramener dans la plage apprise (0.6–0.9)        |
| `prix * 2.5`      | Corriger la sous-estimation des prix du modèle ML | Réalignement avec les prix observés sur Fiverr |


Ces deux ajustements sont pleinement assumés, car ils répondent à une double exigence :

- Cohérence statistique avec le modèle entraîné,
- Crédibilité métier dans les résultats présentés à l'utilisateur final.

Ils constituent une intervention pragmatique pour garantir la fiabilité d’un système hybride entre modélisation et expérience terrain.